In [ ]:
import pandas as pd
import glob

MAGDIR = '/scratch/fl1092/MAG/2021-12-06/' # directory that contains MAG dataset
PROJDIR = '/scratch/fl1092/followup-editors/' # directory that contains the original data pre-anonymized
DATADIR = '../data/' # directory that contains the anonymized data

# Loading data

## MAG data

In [ ]:
def loadPaperAuthor(aff=False):

    columns = ['PaperId', 'AuthorId']
    if aff:
        columns.append('AffiliationId')
        
    papAu = (
        pd.read_csv(MAGDIR+"mag/PaperAuthorAffiliations.txt", sep="\t",
                    names = ['PaperId', 'AuthorId', 'AffiliationId', 'AuthorSequenceNumber',
                             'OriginalAuthor', 'OriginalAffiliation'],
                    usecols = columns,
                    dtype = {'PaperId':int, 'AuthorId':int, 'AffiliationId':float}, memory_map=True)
        .drop_duplicates()
    )
    
    return papAu

def loadRace(threshold=-1, files=[MAGDIR + 'derived/AuthorEthnicity.csv']):
    
    race = (
        pd.concat([pd.read_csv(file, sep='\t', usecols=['AuthorId','Race','RaceScore']) for file in files],
                 sort=False, ignore_index=True)
        .query(f"RaceScore >= {threshold}")
    )
    
    return race

In [ ]:
%%time
paper_journal = pd.read_csv(MAGDIR+'derived/PaperJournal.csv',sep='\t')
paper_year = pd.read_csv(MAGDIR+'derived/PaperYear.csv',sep='\t')

In [ ]:
%%time
papAu = loadPaperAuthor(aff=True)

In [ ]:
%%time
papAuNoAff = loadPaperAuthor(aff=False)

In [ ]:
%%time
oneChina = lambda iso: 'CN' if iso in ['TW', 'HK', 'MO'] else iso

affiliations = (
    pd.read_csv(MAGDIR + "mag/Affiliations.txt", sep="\t",
                names=['AffiliationId', "Rank", "NormalizedName", "DisplayName", "GridId",
                       "OfficialPage", "WikiPage", "PaperCount",'PaperFamilyCount',
                       "CitationCount", "Iso3166Code", "Latitude", "Longitude", 'CreatedDate'],
                usecols=['AffiliationId', 'Iso3166Code', 'NormalizedName'])
    .rename(columns={'Iso3166Code':'iso'})
    .assign(iso = lambda df: df.iso.apply(oneChina))
)
affiliations = affiliations.dropna()

In [ ]:
%%time
affYear = pd.read_csv(MAGDIR + 'derived/AuthorAffiliationYear.csv', sep='\t')

In [ ]:
fieldname = (
    pd.read_csv(MAGDIR + "advanced/FieldsOfStudy.txt", sep="\t", 
                names = ["FieldOfStudyId", "Rank", "NormalizedName", "DisplayName", 
                         "MainType","Level","PaperCount","PaperFamilyCount", "CitationCount","CreatedDate"],
                usecols=['FieldOfStudyId', 'DisplayName']
               )
    .rename(columns={'FieldOfStudyId':'Field', 'DisplayName':'FieldName'})
    .dropna()
)

paperField = (
    pd.read_csv(PROJDIR + 'PaperField.csv', sep='\t', dtype={'PaperId':int}, usecols=['PaperId','Field'])
    .merge(fieldname, on='Field')
)

In [ ]:
nineteenFields = (
    paperField[['FieldName']].drop_duplicates().reset_index(drop=True).reset_index()
    .rename(columns={'index':'FID'})
)
nineteenFields.shape

In [ ]:
nineteenFields.to_csv(DATADIR + 'Fields.tsv', sep='\t', index=False)

In [ ]:
%%time
race = loadRace()

In [ ]:
%%time
authorCareer = pd.read_csv(MAGDIR+'derived/AuthorCareer.csv', sep='\t')

## Paper/editor data from the six publishers

In [ ]:
def loadPaperEditor():
    
    papEditor = (
        pd.read_csv(PROJDIR + 'PaperEditors.csv',sep='\t')
        .drop_duplicates(subset=['PaperId'], keep=False)
    )
    
    return papEditor

In [ ]:
def loadPaperAuthor(aff=False):

    columns = ['PaperId', 'AuthorId']
    if aff:
        columns.append('AffiliationId')
        
    papAu = (
        pd.read_csv(MAGDIR+"mag/PaperAuthorAffiliations.txt", sep="\t",
                    names = ['PaperId', 'AuthorId', 'AffiliationId', 'AuthorSequenceNumber',
                             'OriginalAuthor', 'OriginalAffiliation'],
                    usecols = columns,
                    dtype = {'PaperId':int, 'AuthorId':int, 'AffiliationId':float}, memory_map=True)
        .drop_duplicates()
    )
    
    print(papAu.shape)
    
    return papAu

def loadPaperInfo():
    
    info = pd.read_csv(PROJDIR + 'PaperInfoGathered.csv', sep='\t',
                   usecols=['PaperId','Publisher','Journal'])
    
    return info

def loadJournals():
    
    journals = pd.concat(
        [pd.read_csv(file,sep='\t',dtype={'JournalId':float}).assign(Publisher = file.split('/')[3].lower())
         .dropna().assign(JournalId = lambda df: df.JournalId.astype(int))
         for file in glob.glob('/scratch/fl1092/*/JournalToJournalIdMapping.csv')],
        ignore_index=True, sort=False
    ).drop_duplicates(subset=['JournalId'], keep=False)
    
    return journals

In [ ]:
def loadPaperRecvAcptTime():
    
    recv = (
        pd.read_csv('/scratch/fl1092/followup-editors/RecvTime.csv', sep='\t', parse_dates=['RecvDate'])
        .assign(Year = lambda df: df.RecvDate.apply(lambda x: x.year))
    )

    acpt = (
        pd.read_csv('/scratch/fl1092/followup-editors/AcptTime.csv', sep='\t', parse_dates=['AcptDate'])
        .assign(Year = lambda df: df.AcptDate.apply(lambda x: x.year))
    )
    
    return recv, acpt

In [ ]:
def loadPaperDelay(percentage=True, paperYear=None, normalize=False):
    
    if normalize is True and percentage is True:
        print('both are TURE. Prioritize normalizing instead of percentage.')
    
    if paperYear is None:
        paperYear, _ = loadPaperRecvAcptTime()
        paperYear = paperYear.drop('RecvDate', axis=1)
    
    info = pd.read_csv(PROJDIR + 'PaperInfoGathered.csv', sep='\t',
                   usecols=['PaperId','Publisher','Journal'])
    
    info = info.merge(paperYear, on='PaperId')

    acptDelay = pd.read_csv(PROJDIR + 'AcptDelay.csv', sep='\t', dtype={'AcptDelay':int})

    acptDelay = acptDelay[(acptDelay.AcptDelay > 0) & (acptDelay.AcptDelay <= 730)]

    journalAverage = (
        info.merge(acptDelay, on='PaperId').groupby(['Journal','Year'])
        .AcptDelay.mean().reset_index()
        .rename(columns={'AcptDelay':'JAvg'})
    )
    
    if normalize:
        journalStd = (
            info.merge(acptDelay, on='PaperId').groupby(['Journal','Year'])
            .AcptDelay.std().reset_index()
            .rename(columns={'AcptDelay':'JStd'})
        )

    acptDelay = (
        acptDelay.merge(info, on='PaperId')
        .merge(journalAverage, on=['Journal','Year'])
    )
    
    if normalize:
        acptDelay = (
            acptDelay
            
            .merge(journalStd, on=['Journal', 'Year'])
            .assign(JRelative = lambda df: (df.AcptDelay - df.JAvg)/df.JStd)
        )
    
    elif percentage:
        acptDelay = (
            acptDelay
            
            .assign(JRelative = lambda df: df.AcptDelay - df.JAvg)
            .assign(JRelative = lambda df: df.JRelative/df.JAvg)
        )
    
    return acptDelay

In [ ]:
def loadPaperCountry():
    
    country = (
        
        pd.read_csv('/scratch/fl1092/followup-editors/PaperCountry.csv',sep='\t',
                   dtype={'Percentage':float})
    )
    
    return country

In [ ]:
info = loadPaperInfo()
journals = loadJournals()
paperEditor = loadPaperEditor()

In [ ]:
journalToID = (
    pd.concat([info[['Journal']], journals[['Journal']]], ignore_index=True, sort=False)
    .drop_duplicates().reset_index(drop=True).reset_index()
    .rename(columns={'index':'JID'})
)

In [ ]:
publisherToID = (
    info[['Publisher']].drop_duplicates().reset_index(drop=True).reset_index()
    .rename(columns={'index':'PubID'})
)

In [ ]:
journalToID.to_csv(PROJDIR + "AnonymizedJournalId.csv",sep='\t',index=False)
publisherToID.to_csv(DATADIR + "PublisherId.csv",sep='\t',index=False)

In [ ]:
def mapJournalPublisher(df):
    assert('Publisher' in df.columns)
    assert('Journal' in df.columns)
    
    resdf = (
        df.merge(publisherToID, on='Publisher')
        .merge(journalToID, on='Journal')
        .drop(['Publisher','Journal'], axis=1)
    )
    assert(resdf.shape[0] == df.shape[0])
    
    return resdf

In [ ]:
%%time
paperDelay = loadPaperDelay(percentage=True)

In [ ]:
papCountry = loadPaperCountry()

In [ ]:
%%time
USpapers = (
    pd.read_csv(PROJDIR + 'PaperCountryAll.csv', sep='\t',
                usecols=['PaperId','iso','Percentage'])
    .query('iso == "US"')
    .query('Percentage == 1')
    .drop(['Percentage','iso'], axis=1)
)

In [ ]:
%%time
paperRace = (
    pd.read_csv('/scratch/fl1092/followup-editors/PaperRace.csv', sep='\t')
    .query('Race != "unknown"')
)

In [ ]:
def paperEditorRace(papers, paperEditor=None, race=None, confidence=-1):
    
    if paperEditor is None: paperEditor = loadPaperEditor()
    if race is None: race = loadRace()
        
    papEdiRace = (
        papers.merge(paperEditor, on='PaperId')
        .rename(columns={'EditorId':'AuthorId'})
        .merge(race.query(f'RaceScore > {confidence}'), on='AuthorId', how='left')
        .fillna({'Race':'unknown'})
    )
    
    return papEdiRace

In [ ]:
%%time
editorRace = paperEditorRace(paperDelay[['PaperId']].drop_duplicates(), race=race)

## Auxilary data

In [ ]:
def assignRegion(row):
    
    # create a new region "NA and Oceania"
    
    if row.region == 'Americas':
        if row['sub-region'] == 'Northern America':
            return 'NA and Oceania'
        else:
            return row['sub-region']
        
    elif row.region == 'Oceania': return 'NA and Oceania'
    else:
        return row.region

### map from iso code to country names ###
isoToC = (
    pd.read_csv(DATADIR + 'worldcities.csv',usecols=['iso2','country'])
    .drop_duplicates()
    .rename(columns={'iso2':'iso'})
)
isoToC.loc[isoToC.iso == 'US', 'country'] = 'U.S.A.' # replace country name w/ abbreviation USA
isoToC.loc[isoToC.iso == 'AE', 'country'] = 'U.A.E.' # replace country name w/ abbreviation UAE
isoToC.loc[isoToC.iso == 'GB', 'country'] = 'U.K.' # replace country name w/ abbreviation UK

assert(isoToC.iso.duplicated().any() == False)
assert(isoToC.country.duplicated().any() == False)
### ###

### map from ISO codes to regions 
continents = (
    pd.read_csv(DATADIR + 'continents2.csv',usecols=['iso_3166-2','region','sub-region'])
    .rename(columns={'iso_3166-2':'iso'})
    .assign(iso=lambda df: df.iso.apply(lambda x: x.replace('ISO 3166-2:','')))
    .drop_duplicates()
    .dropna()
    .assign(region=lambda df: df.apply(assignRegion, axis=1))
    .drop('sub-region', axis=1)
)
print(isoToC.shape, continents.shape)
### ###

### color of countries ###
colorsDf = pd.DataFrame({
    'region': ['Africa', 'Latin America and the Caribbean', 'Asia', 'Europe', 'NA and Oceania'], 
    'Color': ['#f34d4d', '#f34d4d', '#f34d4d', '#69bade', '#69bade']}) # white and non-white colors

colors = dict(zip(colorsDf.region, colorsDf.Color))
###
# (223, 2) (248, 2)

In [ ]:
# I have verified with the MAG affiliations file that all the names of institution names correspond
rank = (
    pd.read_csv(DATADIR + 'normaffil_topuniversities_ranking_2019.csv',sep=',',
                usecols=['AffID','rank'], dtype={'AffID':int,'rank':str})
    .rename(columns={'AffID':'AffiliationId'})
)
rank = rank.assign(rank=rank['rank'].apply(lambda x: x.split('-')[0]))
rank = rank.assign(rank=rank['rank'].astype(int))
rank = rank.rename(columns={'rank':'Rank'})
rank.shape, rank['Rank'].max()

# Country representation

In [ ]:
%%time
# authors of papers in our dataset
authors = (
    info
    .merge(paper_year, on='PaperId')
    .merge(papAu, on='PaperId')
    .merge(affiliations, on='AffiliationId')
)

In [ ]:
(
    authors
    .pipe(mapJournalPublisher)
    .to_csv(
        DATADIR + 'country_rep/Authors.tsv', sep='\t', index=False,
        columns=['JID','PubID','iso']
    )
)

In [ ]:
%%time
editors = (
    journals.merge(paper_journal, on='JournalId')
    .drop('JournalId', axis=1)
    .merge(paper_year, on='PaperId')
    
    .merge(paperEditor, on='PaperId')
    .rename(columns={'EditorId': 'AuthorId'})
    .merge(affYear, on=['AuthorId', 'Year'])
    .merge(affiliations, on='AffiliationId')
)

In [ ]:
(
    editors
    .pipe(mapJournalPublisher)
    .to_csv(
        DATADIR + 'country_rep/Editors.tsv', sep='\t', index=False,
        columns=['JID','PubID','iso']
    )
)

## Country representation by field

In [ ]:
%%time
editorField = editors.merge(paperField, on='PaperId')
authorField = authors.merge(paperField, on='PaperId')

In [ ]:
(
    editorField
    .merge(nineteenFields, on='FieldName')
    .to_csv(
        DATADIR + 'country_rep/EditorsField.tsv', sep='\t', index=False,
        columns=['FID','iso']
    )
)

In [ ]:
(
    authorField
    .merge(nineteenFields, on='FieldName')
    .to_csv(
        DATADIR + 'country_rep/AuthorsField.tsv', sep='\t', index=False,
        columns=['FID','iso']
    )
)

In [ ]:
authorFieldTwoPart = (
    authorField.reset_index().rename(columns={'index':'AnoID'})
    .merge(nineteenFields, on='FieldName')
)

In [ ]:
(
    authorFieldTwoPart.head(5251439).to_csv(
        DATADIR + 'country_rep/AuthorsField_0.tsv', sep='\t', index=False,
        columns=['FID','iso']
    )
)

(
    authorFieldTwoPart.tail(5251439).to_csv(
        DATADIR + 'country_rep/AuthorsField_1.tsv', sep='\t', index=False,
        columns=['FID','iso']
    )
)

## Top notch journals

In [ ]:
topNotchJournals = pd.DataFrame([
    ['PLoS_One','plos','multi'],
    ['ijms','mdpi','Biology'],
    ['Front_Microbiol','frontiers','Biology'],
    ['Front_Immunol','frontiers','Biology'],
    ['PNAS','pnas','multi'],
    [6287639, 'ieee','Engineering']],
    columns=['Journal','Publisher','FieldName'])

In [ ]:
topEditors = editors.merge(topNotchJournals, on=['Journal','Publisher'])

topEditorCountryRep = (
    pd.merge(
        topEditors.groupby(['Publisher','Journal','iso']).AuthorId.count().reset_index().rename(columns={'AuthorId':'EdiCount'}),
        topEditors.groupby(['Publisher','Journal']).AuthorId.count().reset_index().rename(columns={'AuthorId':'EdiTotal'}),
        on=['Journal','Publisher']
    )
)

In [ ]:
%%time
authorFieldCountryRep = (
    pd.merge(
        authors.merge(paperField, on='PaperId').groupby(['FieldName','iso']).AuthorId.count().reset_index().rename(columns={'AuthorId':'AutCount'}),
        authors.merge(paperField, on='PaperId').groupby(['FieldName']).AuthorId.count().reset_index().rename(columns={'AuthorId':'AutTotal'}),
        on=['FieldName']
    )
)

authorAllCountryRep = (
    authors
    .groupby('iso').AuthorId.count().reset_index()
    .rename(columns={'AuthorId':'AutCount'})
    .assign(AutTotal=authors.shape[0])
)

authorCountryRep = (
    pd.concat(
        [authorFieldCountryRep,
         authorAllCountryRep.assign(FieldName='multi')],
        ignore_index=True, sort=False
    )
)

In [ ]:
topJournalISO = (
    topNotchJournals.merge(topEditorCountryRep, on=['Publisher','Journal'])
    .merge(authorCountryRep, on=['FieldName','iso'])
)

In [ ]:
topJournalISO.to_csv(DATADIR + 'country_rep/TopJournalCountryRep.tsv',sep='\t',index=False)

# Racial representation

In [ ]:
%%time
# processed in `fig_1_underrepresentation_of_countries.ipynb`

authorsRace = (
    authors
    .drop('NormalizedName', axis=1)
    
    .query('iso == "US"')
    
    .merge(race, on='AuthorId')
    
    .query('Year >= 2001')
    .query('Year <= 2020')
)

editorsRace = (
    editors
    .drop('NormalizedName', axis=1)
    
    .query('iso == "US"')
    
    .merge(race, on='AuthorId')
    
    .query('Year >= 2001')
    .query('Year <= 2020')
)

In [ ]:
assert(authorsRace.drop_duplicates().shape[0] == authorsRace.shape[0])
assert(authorsRace.drop_duplicates().shape[0] == authorsRace.shape[0])

In [ ]:
%%time
(
    authorsRace
    .to_csv(DATADIR + 'race_rep/Authors.tsv',sep='\t',columns=['Publisher','Year','Race'],index=False)
)

(
    editorsRace
    .to_csv(DATADIR + 'race_rep/Editors.tsv',sep='\t',columns=['Publisher','Year','Race'],index=False)
)

## Race representation by field

In [ ]:
ediField = editorsRace.merge(paperField, on='PaperId').query('Year >= 2011').query('Year <= 2020')
autField = authorsRace.merge(paperField, on='PaperId').query('Year >= 2011').query('Year <= 2020')

In [ ]:
%%time
(
    ediField
    .to_csv(DATADIR + 'race_rep/EditorsField.tsv',sep='\t',columns=['FieldName','Year','Race'],index=False)
)

(
    autField
    .to_csv(DATADIR + 'race_rep/AuthorsField.tsv',sep='\t',columns=['FieldName','Year','Race'],index=False)
)

## Top notch journals

In [ ]:
topNotchJournals = pd.DataFrame([
    ['PLoS_One','plos','multi', 'PLOS One'],
    ['ijms','mdpi','Biology', 'Int. J. Mol. Sci.'],
    ['Front_Microbiol','frontiers','Biology', 'Front. Microbiol.'],
    ['Front_Immunol','frontiers','Biology', 'Front. Immunol.'],
    ['PNAS','pnas','multi', 'PNAS'],
    [6287639, 'ieee','Engineering', 'IEEE Access']],
    columns=['Journal','Publisher','FieldName','JournalName'])
topNotchJournals.shape

In [ ]:
ediTop = (
    editorsRace
    .merge(topNotchJournals, on=['Journal', 'Publisher'])
)

In [ ]:
autTop = (
    pd.concat([
        autField.drop(['Journal','Publisher'], axis=1)
        .merge(topNotchJournals, on=['FieldName']),
        autField.assign(JournalName='PLOS One').assign(Publisher='PLOS').assign(FieldName='multi'),
        autField.assign(JournalName='PNAS').assign(Publisher='PNAS').assign(FieldName='multi'),
    ], ignore_index=True)
)

In [ ]:
ediTop.to_csv(
    DATADIR + 'race_rep/EditorsTopJournals.tsv',sep='\t',index=False,
    columns=['JournalName','Race','Year']
)

autTop.to_csv(
    DATADIR + 'race_rep/AuthorsTopJournals.tsv',sep='\t',index=False,
    columns=['JournalName','Race','Year']
)

# Acceptance delay by country

In [ ]:
delay = (
    paperDelay.merge(papCountry, on='PaperId')
    .merge(isoToC, on='iso')
    .merge(continents, on='iso')
)

In [ ]:
paperByCountry = (
    delay.groupby(['country','iso']).PaperId.nunique().reset_index()
    .rename(columns={'PaperId':'Count'})
)
paperByCountry.to_csv(DATADIR + 'acpt_delay/CountryPaperCount.tsv',sep='\t',index=False)

In [ ]:
delay.to_csv(
    DATADIR + 'acpt_delay/PaperCountryDelay.tsv',sep='\t',index=False,
    columns=['Publisher','Year','JRelative','iso','country','region']
)

## By Field

In [ ]:
fieldDelay = (
    delay
    .merge(colorsDf, on='region')
    .merge(paperField, on='PaperId')
)

In [ ]:
fieldDelay.to_csv(
    DATADIR + 'acpt_delay/PaperFieldDelay.tsv',sep='\t',index=False,
    columns=['Year','FieldName','Color','JRelative']
)

## Interaction

In [ ]:
sameCountry = pd.read_csv(PROJDIR + 'PaperEditorAuthorSameCountry.csv',sep='\t')
sameCountry.shape, sameCountry.PaperId.nunique()

In [ ]:
toPlotDelayAll = pd.read_csv(DATADIR + 'acpt_delay/Countries.tsv', sep='\t')
interactionCountries = toPlotDelayAll[['iso']].drop_duplicates()

In [ ]:
interaction = (
    interactionCountries.merge(delay, on='iso')
    .merge(sameCountry, on='PaperId')
)

In [ ]:
interaction.to_csv(
    DATADIR + 'acpt_delay/CountryInteraction.tsv',sep='\t',index=False,
    columns=['iso','country','SameCountry','JRelative']
)

# Acceptance delay by race

In [ ]:
%%time
raceDelay = (
    USpapers.merge(paperDelay, on='PaperId')
    
    .merge(paperRace, on='PaperId')
    
    .query('Year >= 2000')
)

In [ ]:
raceDelay.to_csv(DATADIR + 'acpt_delay/PaperRaceDelay.tsv',sep='\t',index=False,columns=['Year','Race','JRelative'])

## Interaction

In [ ]:
%%time
interaction = (
    paperDelay[['PaperId','JRelative']]
    .merge(USpapers, on='PaperId')
    
    .merge(
        editorRace
        .query('Race != "unknown"')
        .rename(columns={'Race':'EditorRace'})
        [['PaperId','EditorRace']], on='PaperId'
    )
    .merge(
        paperRace
        .query('Race != "unknown"')
        .rename(columns={'Race':'PaperRace'})
        [['PaperId','PaperRace']], on='PaperId'
    )
    .assign(PaperIsWhite=lambda df: df.PaperRace.apply(lambda x: x=='White'))
    .assign(EditorIsWhite=lambda df: df.EditorRace.apply(lambda x: x=='White'))
    
    .assign(PaperIsBlack=lambda df: df.PaperRace.apply(lambda x: x=='Black'))
    .assign(EditorIsBlack=lambda df: df.EditorRace.apply(lambda x: x=='Black'))
    
    .assign(SameRace=lambda df: df.apply(lambda row: row['PaperRace'] == row['EditorRace'], axis=1))
)

In [ ]:
interaction.to_csv(
    DATADIR + 'acpt_delay/RaceInteraction.tsv',sep='\t',index=False,
    columns=['PaperIsWhite','EditorIsWhite','JRelative']
)

## Regression

### Compute paper attributes

In [ ]:
%%time
paperAuthor = papAuNoAff.merge(paperDelay[['PaperId']].drop_duplicates(), on='PaperId')
paperAuthorAff = papAu.merge(paperDelay[['PaperId']].drop_duplicates(), on='PaperId')

In [ ]:
%%time
paperAuthorCount = (
    paperAuthor.groupby('PaperId').AuthorId.nunique()
    .reset_index().rename(columns={'AuthorId':'Total'})
)

In [ ]:
%%time
paperRaceCount = (
    paperAuthor.merge(race[race.Race.isin(['White', 'Hispanic', 'API', 'Black'])], on='AuthorId')
    
    .groupby(['PaperId','Race']).AuthorId.nunique().reset_index()
    .rename(columns={'AuthorId':'RaceCount'})
    .merge(paperAuthorCount, on='PaperId')
    .assign(RacePercent = lambda df: df.apply(lambda row: row.RaceCount/row.Total, axis=1))
)

In [ ]:
%%time
paperCountryCount = (
    paperAuthorAff.merge(affiliations, on='AffiliationId')
    
    .groupby(['PaperId','iso']).AuthorId.nunique().reset_index()
    .rename(columns={'AuthorId':'CountryCount'})
    
    .merge(paperAuthorCount, on='PaperId')
    .assign(CountryPercent = lambda df: df.apply(lambda row: row.CountryCount/row.Total, axis=1))
)

In [ ]:
paperCountryPivot = (
    paperCountryCount.pivot(index='PaperId', columns='iso', values='CountryCount').fillna(0).reset_index()
)

paperRacePivot = (
    paperRaceCount.pivot(index='PaperId', columns='Race', values='RaceCount').fillna(0).reset_index()
)

In [ ]:
%%time
paperAffRank = (
    paperAuthorAff.merge(rank, on='AffiliationId', how='left').fillna({'Rank':1001})
    .groupby(['PaperId']).Rank.min().reset_index()
    .assign(Rank = lambda df: df.Rank.astype(int))
) # 1000990

In [ ]:
%%time
paperAge = (
    paperAuthor.merge(paper_year, on='PaperId')
    .merge(authorCareer, on='AuthorId')
    .assign(Age = lambda df: df.Year - df.Yfp)
    .groupby('PaperId').Age.max().reset_index()
) # 1000989

#### Editor country and race

In [ ]:
%%time
editorAff = (
    pd.read_csv(PROJDIR + 'EditorAffiliationYear.csv', sep='\t', usecols=['AuthorId','Year','iso'])
    .drop_duplicates()
)

In [ ]:
%%time
paperEditorCountry = (
    paperEditor.rename(columns={'EditorId':'AuthorId'})
    .merge(paper_year, on='PaperId')
    .merge(editorAff, on=['AuthorId','Year'])
    .rename(columns={'AuthorId':'EditorId', 'iso':'EditorIso'})
)

In [ ]:
%%time
sameCountry = (
    paperEditorCountry.merge(paperCountryCount, on='PaperId')
    
    .assign(Same = lambda df: df.iso == df.EditorIso)
    .groupby('PaperId').Same.any().reset_index()
    .rename(columns={'Same':'SameCountry'})
)

In [ ]:
%%time
paperEditorRace = (
    paperEditor.rename(columns={'EditorId':'AuthorId'}).merge(race, on='AuthorId')
    .pipe(printShape)
    
    .rename(columns={'AuthorId':'EditorId', 'Race':'EditorRace'})
)

In [ ]:
%%time
sameRace = (
    paperEditorRace.merge(paperRaceCount, on='PaperId')
    
    .assign(Same = lambda df: df.Race == df.EditorRace)
    .groupby('PaperId').Same.any().reset_index()
    .rename(columns={'Same':'SameRace'})
)

In [ ]:
%%time
editorRaceCountry = (
    paperDelay[['PaperId']].drop_duplicates()
    .merge(sameRace, on='PaperId', how='left')
    .merge(sameCountry, on='PaperId', how='left')
    .fillna({'SameRace':False, 'SameCountry':False})
) 

#### Paper field

In [ ]:
fieldname = (
    pd.read_csv(MAGDIR + "advanced/FieldsOfStudy.txt", sep="\t", 
                names = ["FieldOfStudyId", "Rank", "NormalizedName", "DisplayName", 
                         "MainType","Level","PaperCount","PaperFamilyCount", "CitationCount","CreatedDate"],
                usecols=['FieldOfStudyId', 'DisplayName']
               )
    .rename(columns={'FieldOfStudyId':'Field', 'DisplayName':'FieldName'})
    .dropna()
    .assign(FieldName = lambda df: df.FieldName.apply(lambda x: '_'.join(x.split())))
)

paperField = (
    pd.read_csv(PROJDIR + 'PaperField.csv', sep='\t', dtype={'PaperId':int}, usecols=['PaperId','Field'])
    .merge(fieldname, on='Field')
    .drop('Field', axis=1)
)


oneField = paperField.drop_duplicates(subset='PaperId', keep=False)

paperField = (
    pd.concat([oneField,
               (
                   paperField[~paperField.PaperId.isin(oneField.PaperId)]
                   .assign(FieldName='multi')
                   .drop_duplicates()
               )], ignore_index=True, sort=False)
)

In [ ]:
paperFieldPivot = (
    paperField.assign(Value=1).pivot(index='PaperId',columns='FieldName',values='Value')
    .fillna(0).reset_index()
)

### Gather paper attributes

In [ ]:
%%time
paperDf = (
    paperDelay[['PaperId','JRelative']].drop_duplicates()
    .merge(paper_year, on='PaperId')
    .merge(paperAuthorCount, on='PaperId')
    .merge(paperRacePivot, on='PaperId')
    .merge(paperCountryPivot, on='PaperId')
    .merge(paperAffRank, on='PaperId')
    .merge(paperAge, on='PaperId')
    .merge(editorRaceCountry, on='PaperId')
    .merge(paperFieldPivot, on='PaperId')
)

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
%%time
for ind, df in tqdm(enumerate(np.array_split(paperDf, 10))):
    df.drop(['PaperId'],axis=1).to_csv(DATADIR + f'acpt_delay/regression_data/{ind}.tsv',sep='\t',index=False)

In [ ]:
paperDf.columns.values

In [ ]:
country_control = ' + '.join([x for x in paperCountryCount.iso.unique() if x != 'US']) # relative to US

In [ ]:
with open(DATADIR + 'CountryControl.txt', 'w+') as f:
    f.write(country_control)